In [ ]:
dataset_img = PathMNIST(split="train", size=224, download=True)

In [ ]:
plt.pcolormesh(dataset_img[7][0])
print(len(dataset_img))

In [ ]:
class DiseaseDataset(Dataset):
    def __init__(self, helperDataset):
        
        newDatasetInput = []
        newDatasetOutput = []
        i = 0
        with torch.no_grad():
            for inp, out in helperDataset:
                i += 1
                inputs = processor(inp, return_tensors="pt").to(device)
                outputs = model(**inputs)
        
                newDatasetInput.append(torch.mean(outputs.last_hidden_state, axis=1)[0].cpu())
                
                newDatasetOutput.append(out[0])
                if i%1000==0:
                    print("Yes")

        self.newDatasetInput = torch.tensor(np.array(newDatasetInput)).to(device)
        self.newDatasetOutput = torch.tensor(np.array(newDatasetOutput), dtype=torch.int64).to(device)
        print(self.newDatasetInput.shape[0])

    def __len__(self):
        return self.newDatasetInput.shape[0]

    def __getitem__(self, idx):
        embedding = self.newDatasetInput[idx]
        label = self.newDatasetOutput[idx]

        return embedding, label

In [ ]:
import torch
import os

def save_disease_dataset(dataset, save_dir):
    """
    Saves the processed inputs and outputs of a DiseaseDataset instance.

    Args:
        dataset (DiseaseDataset): Instance of DiseaseDataset to save.
        save_dir (str): Directory where to save the files.
    """
    os.makedirs(save_dir, exist_ok=True)

    input_path = os.path.join(save_dir, "newDatasetInput.pt")
    output_path = os.path.join(save_dir, "newDatasetOutput.pt")
    
    torch.save(dataset.newDatasetInput.cpu(), input_path)
    torch.save(dataset.newDatasetOutput.cpu(), output_path)
    
    print(f"Saved input embeddings to {input_path}")
    print(f"Saved output labels to {output_path}")